In [2]:
## data generation
import sys, os
import matplotlib.pyplot as plt
sys.path.insert(0, os.getcwd())



In [19]:
pip install optax==0.1.7


  Attempting uninstall: optax
    Found existing installation: optax 0.1.8
    Uninstalling optax-0.1.8:
      Successfully uninstalled optax-0.1.8
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 3.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
!python3 -m venv .venv
!source .venv/bin/activate      # (Windows PowerShell: .\.venv\Scripts\Activate.ps1)
!pip install numpy


In [6]:
pip install -U jax jaxlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.8 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 2.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.5 MB/s eta 0:00:00a 0:00:01
  Created wheel for jax: filename=jax-0.4.13-py3-none-any.whl size=1518704 sha256=dbbf3d3b2dbd69848959967f299880ebf4e58e8b674b4b5c360ba5b778de3c51
  Stored in directory: /Users/banshihan/Library/Caches/pip/wheels/46/d9/15/d2800d4089dc4c77299ac7513c6aa1036f5491edbd2bf6ba16
Successfully built jax
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install flax


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 2.9 MB/s eta 0:00:00a 0:00:01m
  Created wheel for msgpack: filename=msgpack-1.1.0-cp38-cp38-macosx_11_0_arm64.whl size=83182 sha256=9112e7a38080ae0a7a091a1b5f45fd30e3db31545714ee284be90eb489cdab0a
  Stored in directory: /Users/banshihan/Library/Caches/pip/wheels/55/aa/93/797450f0b3d3ac6906a2a32306efbb304940a5a8eb5bdff767
Successfully built msgpack
Note: you may need to restart the kernel to use updated packages.


## Linear Case

## Linear case data preparation

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm


In [2]:
!pwd  # 看你当前在哪
!ls   # 确保能看到 seed_linpadding_expts.sh 和 run.py


/Users/banshihan/Documents/GitHub/homework-3-ShihanBan/Diffusion-model-low-dimensional-data
__init__.py              diffusion.py             networks.py
__pycache__              diffusion_model.py       run.py
data                     diffusion_utils.py       seed_linpadding_expts.sh
data_generation.ipynb    generated_data           training.py
datasets.py              model.py                 utils.py


In [3]:
# Notebook Cell 1: 导入
import numpy as np
from datasets import LinearGaussianDataset    # 原仓库中的类定义 :contentReference[oaicite:0]{index=0}



In [4]:
import numpy as np
from datasets import LinearGaussianDataset

# 固定 intrinsic dimension r* = 3
r_star = 3  

# 定义 ambient 维度到 padding_dim 的映射（对应脚本里的组合）
dd_to_paddings = {
    3: [9, 17],
    6: [6, 14],
    9: [3, 11],
    12:[8],      # 脚本里只对 12 维做了一个 padding_dim 组合
}

# 三个随机种子 ds ∈ {2,3,4}
seeds = [2, 3, 4]

# 样本数 N = num_batches * batch_size = 100000 * 128
N = 100_000 * 128  

# 存放所有生成结果的字典
data_store = {}

for dd, paddings in dd_to_paddings.items():
    for pad in paddings:
        for seed in seeds:
            key = f"d{dd}_pad{pad}_seed{seed}"
            # 实例化 Dataset
            ds = LinearGaussianDataset(
                seed=seed,
                dimension=dd,
                intrinsic_dimension=r_star,
                padding_dimension=pad,
                var_added=0.0
            )
            # 一次性生成 N 个样本
            X = ds.get_batch(N)  # jax.numpy array of shape (N, dd+pad)
            # 转为 NumPy 并存储
            data_store[key] = np.array(X)
            print(f"{key} → shape = {data_store[key].shape}")



d3_pad9_seed2 → shape = (12800000, 12)
d3_pad9_seed3 → shape = (12800000, 12)
d3_pad9_seed4 → shape = (12800000, 12)
d3_pad17_seed2 → shape = (12800000, 20)
d3_pad17_seed3 → shape = (12800000, 20)
d3_pad17_seed4 → shape = (12800000, 20)
d6_pad6_seed2 → shape = (12800000, 12)
d6_pad6_seed3 → shape = (12800000, 12)
d6_pad6_seed4 → shape = (12800000, 12)
d6_pad14_seed2 → shape = (12800000, 20)
d6_pad14_seed3 → shape = (12800000, 20)
d6_pad14_seed4 → shape = (12800000, 20)
d9_pad3_seed2 → shape = (12800000, 12)
d9_pad3_seed3 → shape = (12800000, 12)
d9_pad3_seed4 → shape = (12800000, 12)
d9_pad11_seed2 → shape = (12800000, 20)
d9_pad11_seed3 → shape = (12800000, 20)
d9_pad11_seed4 → shape = (12800000, 20)
d12_pad8_seed2 → shape = (12800000, 20)
d12_pad8_seed3 → shape = (12800000, 20)
d12_pad8_seed4 → shape = (12800000, 20)


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import math

# 1. 时间嵌入模块
class SinusoidalPositionEmbeddings(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
    
    def forward(self, t):
        device = t.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = t[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings

# 2. 扩散模型网络结构
class DiffusionModel(nn.Module):
    def __init__(self, input_dim, time_dim=32):
        super().__init__()
        self.time_mlp = nn.Sequential(
            SinusoidalPositionEmbeddings(time_dim),
            nn.Linear(time_dim, time_dim),
            nn.SiLU(),
            nn.Linear(time_dim, time_dim),
        )
        self.input_mlp = nn.Sequential(
            nn.Linear(input_dim + time_dim, 256),
            nn.SiLU(),
            nn.Linear(256, 256),
            nn.SiLU(),
            nn.Linear(256, input_dim),
        )
    
    def forward(self, x, t):
        t_emb = self.time_mlp(t)
        x = torch.cat([x, t_emb], dim=1)
        return self.input_mlp(x)

# 3. 扩散过程管理类
class DiffusionProcess:
    def __init__(self, noise_steps=1000, beta_start=1e-4, beta_end=0.02, device='cuda'):
        self.noise_steps = noise_steps
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.device = device
        
        # 预计算扩散计划参数
        self.beta = torch.linspace(beta_start, beta_end, noise_steps, device=device)
        self.alpha = 1. - self.beta
        self.alpha_bar = torch.cumprod(self.alpha, dim=0)
    
    def sample_timesteps(self, batch_size):
        return torch.randint(0, self.noise_steps, (batch_size,), device=self.device)
    
    def add_noise(self, x, t):
        sqrt_alpha_bar = torch.sqrt(self.alpha_bar[t])[:, None]
        sqrt_one_minus_alpha_bar = torch.sqrt(1 - self.alpha_bar[t])[:, None]
        noise = torch.randn_like(x)
        return sqrt_alpha_bar * x + sqrt_one_minus_alpha_bar * noise, noise
    
    def denoise_step(self, model, x, t):
        with torch.no_grad():
            pred_noise = model(x, t)
            alpha = self.alpha[t][:, None]
            alpha_bar = self.alpha_bar[t][:, None]
            beta = self.beta[t][:, None]
            
            x = (x - (beta / torch.sqrt(1 - alpha_bar)) * pred_noise) / torch.sqrt(alpha)
            if t[0] > 0:  # 添加过程噪声
                x += torch.sqrt(beta) * torch.randn_like(x)
        return x
    def sample(self, model, num_samples, input_dim, device='cuda'):
        model.eval()
        with torch.no_grad():
            # 初始噪声
            x = torch.randn((num_samples, input_dim)).to(device)
            
            # 反向扩散过程
            for t in reversed(range(self.noise_steps)):
                t_batch = torch.full((num_samples,), t, device=device)
                x = self.denoise_step(model, x, t_batch)
        return x.cpu().numpy()

def compute_metrics(model, original_data, generated_data, device='cuda'):
    metrics = {}
    
    # 指标1: 本征维度计算（使用模型参数）
    weights = []
    for param in model.parameters():
        if len(param.shape) == 2:  # 只处理权重矩阵
            weights.append(param.detach().cpu().numpy())
    W = np.concatenate([w.flatten() for w in weights])
    U, s, Vt = svd(W.reshape(-1, W.shape[0]), full_matrices=False)
    explained_variance = np.cumsum(s**2) / np.sum(s**2)
    intrinsic_dim = np.argmax(explained_variance >= 0.95) + 1
    metrics['intrinsic_dim'] = intrinsic_dim
    
    # 指标2: 解码器非零行平均数
    decoder_weights = model.input_mlp[0].weight.detach().cpu().numpy()
    threshold = 1e-5  # 非零阈值
    nonzero_rows = np.mean([np.any(np.abs(row) > threshold) for row in decoder_weights])
    metrics['decoder_nonzero_rows'] = nonzero_rows

    # 指标3: 归一化特征值误差
    # 原始数据协方差
    orig_cov = np.cov(original_data.T)
    orig_eigvals = np.linalg.eigvalsh(orig_cov)
    
    # 生成数据协方差
    gen_cov = np.cov(generated_data.T)
    gen_eigvals = np.linalg.eigvalsh(gen_cov)
    
    # 计算归一化误差
    error = np.linalg.norm(orig_eigvals - gen_eigvals) / np.linalg.norm(orig_eigvals)
    metrics['normalized_eigen_error'] = error
    
    return metrics

# 5. 训练函数
def train_diffusion_model(data_dict, device='cuda', num_epochs=10, batch_size=128, lr=1e-3):
    all_metrics = {}
    
    for key in tqdm(data_dict, desc="Datasets"):
        # 数据准备
        data_np = data_dict[key]
        input_dim = data_np.shape[1]
        dataset = TensorDataset(torch.tensor(data_np).float())
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
        
        # 模型初始化
        model = DiffusionModel(input_dim).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        diffusion = DiffusionProcess(device=device)
        
        # 训练循环
        tqdm.write(f"\n=== Training {key} (dim={input_dim}) ===")
        for epoch in range(num_epochs):
            model.train()
            total_loss = 0
            for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
                x = batch[0].to(device)
                
                # 扩散过程
                t = diffusion.sample_timesteps(x.size(0))
                x_noisy, true_noise = diffusion.add_noise(x, t)
                
                # 优化步骤
                optimizer.zero_grad()
                pred_noise = model(x_noisy, t)
                loss = nn.MSELoss()(pred_noise, true_noise)
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item() * x.size(0)
            
            # 打印训练信息
            avg_loss = total_loss / len(dataloader.dataset)
            tqdm.write(f"Epoch {epoch+1}/{num_epochs} | Loss: {avg_loss:.6f}")
        
        # 保存模型
        torch.save(model.state_dict(), f"diffusion_{key}.pth")
        
        # 生成样本并评估
        generated_data = diffusion.sample(model, 5000, input_dim, device)
        metrics = compute_metrics(model, data_np, generated_data)
        all_metrics[key] = metrics
        
        # 保存评估结果
        with open(f"metrics_{key}.txt", "w") as f:
            f.write(f"Intrinsic Dimension: {metrics['intrinsic_dim']}\n")
            f.write(f"Decoder Nonzero Rows: {metrics['decoder_nonzero_rows']:.4f}\n")
            f.write(f"Normalized Eigen Error: {metrics['normalized_eigen_error']:.4f}\n")
        
        tqdm.write(f"\nEvaluation for {key}:")
        tqdm.write(f"Intrinsic Dim: {metrics['intrinsic_dim']}")
        tqdm.write(f"Nonzero Rows: {metrics['decoder_nonzero_rows']:.4f}")
        tqdm.write(f"Eigen Error: {metrics['normalized_eigen_error']:.4f}\n")
    
    return all_metrics


In [9]:
# 5. 执行训练

if __name__ == "__main__":
    # ... 保持原有代码不变 ...
    
    # 开始训练并获取指标
    metrics = train_diffusion_model(
        data_store,
        device=device,
        num_epochs=10,
        batch_size=256,
        lr=1e-3
    )
    
    # 打印汇总表格
    print("\nFinal Metrics Summary:")
    print(f"{'Dataset':<20} | {'Intrinsic Dim':<15} | {'Nonzero Rows':<15} | {'Eigen Error':<15}")
    for key in metrics:
        m = metrics[key]
        print(f"{key:<20} | {m['intrinsic_dim']:<15} | {m['decoder_nonzero_rows']:<15.4f} | {m['normalized_eigen_error']:<15.4f}")

Datasets:   0%|          | 0/21 [00:00<?, ?it/s]


=== Training d3_pad9_seed2 (dim=12) ===


Datasets:   0%|          | 0/21 [04:27<?, ?it/s]

Epoch 1/10 | Loss: 0.087585


Datasets:   0%|          | 0/21 [06:55<?, ?it/s]


KeyboardInterrupt: 